# Scraping League of Legends mains subreddits

Ideas - Do document clustering by subreddits, get most important words for each cluster and each subreddit (n_clusters = # of roles in league)

Try clustering by top all time, and top monthly 

See corellation between post activity and win percentage (scrape op.gg)

Create network of people who post in multiple subreddits

compare the average upvote ratio for each subreddit 

compare the videos to non-videos posted

Spot shitposting - figure out how later

Plot activity by date of release and by date of last update 

In [166]:
# Create a list of champions - to be used for subreddits URLs
# There are some special cases, which will be explained next cell...

mains = ["Aatrox","Ahri","Akali","Amumu","Anivia","Annie","Ashe","Aurelion_Sol_",
         "Azir","Bard",

"Blitzcrank",

"Brand",

"Braum",

"Caitlyn",

"Camille",

"Cassiopeia",

"ChoGath",

"Corki",

"Darius",

"Diana",

"DrMundo",

"Ekko",

"Elise",

"Evelynn",

"Ezreal",

"Fiddlesticks",

"Fiora",

"Fizz",

"Galio",

"Gangplank",

"Garen",

"Gnar",

"Gragas",

"Graves",

"Hecarim",

"Heimerdinger",

"Irelia",

"Ivern",

"JarvanIV",

"Jax",

"Jayce",

"Jhin",

"Kalista",

"Karma",

"Karthus",

"Kassadin",

"Katarina",

"Kayle",

"Kayn",

"Kennen",

"KhaZix",


"Kled",

"KogMaw",

"LeBlanc",

"LeeSin",

"Leona",

"Lissandra",

"Lucian",

"Lulu",

"Malphite",

"Malzahar",

"Maokai",

"MasterYi",

"MissFortune",

"Mordekaiser",

"Morgana",

"Nami",

"Nasus",

"Nautilus",

"Nidalee",

"Nocturne",

"Nunu",

"Olaf",

"Orianna",

"Ornn",

"Pantheon",

"Poppy",

"Quinn",

"Rakan",

"Rammus",

"RekSai",

"Renekton",

"Rengar",

"Riven",

"Rumble",

"Ryze",

"Sejuani",

"Shaco",

"Shyvana",

"Singed",

"Skarner",

"Sona",

"Soraka",

"Swain",

"Syndra",

"TahmKench",

"Taliyah",

"Talon",

"Taric",

"Thresh",

"Tristana",

"Trundle",

"Tryndamere",

"TwistedFate",

"Twitch",

"Udyr",

"Urgot",

"Varus",

"Vayne",

"Veigar",

"Vi",

"Viktor",

"Vladimir",

"Volibear",

"Warwick",

"Wukong",

"Xayah",

"Xerath",

"XinZhao",

"Yasuo",

"Yorick",

"Zed",

"Ziggs",

"Zilean",

"Zoe",

"Zyra"]

In [167]:
mains_with_reddit = []
for i in range(len(mains)):
    mains_with_reddit.append(str(mains[i])+'mains')
# mains_with_reddit

In [168]:
# One of these mains is not like the others...
# Most main subreddits are in the form *champname* + Mains
# Some are not...here are the ones that are not in that format

# Zac - r/thesecretweapon
# Velkoz - r/velkoz 
# Teemo - r/TeemoTalk
# Sivir - r/Sivir
# Sion - r/DirtySionMains
# Shen - r/Shen
# Lux - r/Lux
# Kindred - r/Kindred
# Jinx - r/leagueofjinx 
# Janna - r/Janna
# Illoia - r/Illaoi
# Draven - r/Draven 
# A sol mains - r/Aurelion_Sol_mains -- already in list, kinda works

main_sub_names = mains_with_reddit

main_sub_names.append('thesecretweapon')
main_sub_names.append('velkoz')
main_sub_names.append('TeemoTalk')
main_sub_names.append('Sivir')
main_sub_names.append('DirtySionMains')
main_sub_names.append('Shen')
main_sub_names.append('Lux')
main_sub_names.append('Kindred')
main_sub_names.append('leagueofjinx')
main_sub_names.append('Janna')
main_sub_names.append('Illaoi')
main_sub_names.append('Draven')
main_sub_names.append('Alistar')

# main_sub_names

# Start the scraping 
we have all the URL's we need! now the hard part...

Try using PRAW

In [2]:
import praw

reddit = praw.Reddit(client_id='F2XQbVxMX5Al6w',
                     client_secret='GD0yT4BQvaeF--bzcJVCBCyrVA8',
                     password='slack298l',
                     user_agent='Lol Main subreddit scrapes by u/slackerlol',
                     username='slackerlol')

print(reddit.user.me()) #check if you got in

slackerlol


In [82]:
subreddit = reddit.subreddit('dirtysionmains')
top_comments = []

for submission in subreddit.top(limit=1):
    print(submission.title)  # Output: the submission's title
    print(submission.score)  # Output: the submission's score
    print(submission.id)     # Output: the submission's ID
    print(submission.url)    # Output: the URL the submission points to
    
#     top_level_comments = list(submission.comments)
#     for top_level_comment in submission.comments:
    submission.comments.replace_more(limit=None)
    for comment in submission.comments.list():
        print(comment.body)

    print()

Sion Q move
113
7an9go
https://www.youtube.com/watch?v=LJYSjHm5up8
Quality fuckin shit post, 5/7 my dude
A masterpiece
Nice
i love this shit
7/7 with rice.
So you're a cultist if 5/7 too?



In [143]:
import pprint

# assume you have a Reddit instance bound to variable `reddit`
submission = reddit.submission(id='5vfjww')
print(submission.title) # to make it non-lazy
# pprint.pprint(vars(submission))
(vars(submission))

In [144]:
# Define function to return the metrics
import numpy as np
import pandas as pd

def pull_sub_data(subreddit, lim):

    subreddit = reddit.subreddit(subreddit)
    
    titles = []
    scores = []
    post_ids = []
    post_urls = []
    comments_all = []
    types = []
    upvote_ratios = []
    authors = []

    for submission in subreddit.top(limit=lim):
        title = submission.title  # Output: the submission's title
        score = submission.score  # Output: the submission's score
        post_id = submission.id   # Output: the submission's ID
        post_url = submission.url   # Output: the URL the submission points to
        author = submission.author 
        
        # Append lists
        titles.append(title)
        scores.append(score)
        post_ids.append(post_id)
        post_urls.append(post_url)
        authors.append(author)

        # Get comments
        comments = []
        submission.comments.replace_more(limit=None)
        for comment in submission.comments.list():
            comment_app = {comment.author:comment.body} # Get author and content - in dict form
            comments.append(comment_app)
            
        comments_all.append(comments)

        # Get is_video and upvote ratio
        upvote_ratio = vars(submission)['upvote_ratio']
        upvote_ratios.append(upvote_ratio)
        
        try:
            type_post = vars(submission)['media']['type']
            types.append(type_post)
        except:
            if vars(submission)['is_reddit_media_domain'] == True:
                types.append('reddit_media')
            if vars(submission)['is_self'] == True:
                types.append('self_post')
            else:
                types.append('unknown')
        

        
    df = pd.DataFrame(list(zip(titles, authors, scores, post_ids, post_urls, comments_all, types,
                              upvote_ratios)), columns=['title', 'author', 'score', 'post_id','post_url',
                                                       'comments','type','upvote_ratio'])
    
    return df


In [ ]:
# pull_sub_data('dirtysionmains', 2)
# sion_test = pull_sub_data('dirtysionmains', 2)
# sion_test.to_csv('sion_test.csv')

# Ok!  So we got the first version of our text scrape written!
Now to run through and do the actual scraping

In [165]:
import time


for champ_main in main_sub_names:
    attempts = 0
    # Sometimes we get a bad connection, so use try/except for 5 loops - if it fails after third
    # then just pass and let it be known
    while True and attempts <= 5:
        try:
            subreddit_df = pull_sub_data(champ_main, 50)
        except:
            print(attempts, 'attempt for', champ_main, 'failed; retrying...')
            time.sleep(5)
            continue
        break

    subreddit_df.to_csv(champ_main+'_df.csv')

0 attempt for Aatroxmains failed; retrying...
0 attempt for Elisemains failed; retrying...
0 attempt for Singedmains failed; retrying...
0 attempt for Tahm Kenchmains failed; retrying...
0 attempt for Tahm Kenchmains failed; retrying...
0 attempt for Tahm Kenchmains failed; retrying...
0 attempt for Tahm Kenchmains failed; retrying...
0 attempt for Tahm Kenchmains failed; retrying...
0 attempt for Tahm Kenchmains failed; retrying...
0 attempt for Tahm Kenchmains failed; retrying...
0 attempt for Tahm Kenchmains failed; retrying...
0 attempt for Tahm Kenchmains failed; retrying...
0 attempt for Tahm Kenchmains failed; retrying...
0 attempt for Tahm Kenchmains failed; retrying...
0 attempt for Tahm Kenchmains failed; retrying...
0 attempt for Tahm Kenchmains failed; retrying...
0 attempt for Tahm Kenchmains failed; retrying...
0 attempt for Tahm Kenchmains failed; retrying...
0 attempt for Tahm Kenchmains failed; retrying...
0 attempt for Tahm Kenchmains failed; retrying...
0 attempt for

KeyboardInterrupt: 

In [172]:
# Crashed on tkmains - restart from here manually 
main_sub_names.index('TahmKenchmains')

95

In [176]:
temp_sub_names = main_sub_names[95:]

In [177]:
# for champ_main in temp_sub_names:
#     attempts = 0
#     # Sometimes we get a bad connection, so use try/except for 5 loops - if it fails after third
#     # then just pass and let it be known
#     while True and attempts <= 5:
#         try:
#             subreddit_df = pull_sub_data(champ_main, 50)
#         except:
#             print(attempts, 'attempt for', champ_main, 'failed; retrying...')
#             time.sleep(5)
#             continue
#         break

#     subreddit_df.to_csv(champ_main+'_df.csv')